In [1]:
import cifar10
import matplotlib.pyplot as plt

In [2]:
cifar10.data_patch='data/Cifar-10/'

In [3]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [4]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [5]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [6]:
print("images_train",images_train.shape)
print("cls_train",cls_train.shape)
print("labels_train",labels_train.shape)

images_train (50000, 32, 32, 3)
cls_train (50000,)
labels_train (50000, 10)


In [7]:
print("images_test",images_test.shape)
print("cls_test",cls_test.shape)
print("labels_test",labels_test.shape)

images_test (10000, 32, 32, 3)
cls_test (10000,)
labels_test (10000, 10)


In [8]:
train_pixel=images_train.reshape(50000,3072)
test_pixel=images_test.reshape(10000,3072)

In [9]:
X_train,X_test,Y_train,Y_test=train_pixel,test_pixel,labels_train,labels_test

In [10]:
X_train=X_train/255
X_test=X_test/255

In [11]:
k=3072

In [12]:
from keras.layers import Dense
from keras.models import Sequential

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
model=Sequential()

In [14]:
layer1=Dense(units=512,activation='relu',input_dim=k)
model.add(layer1)
layer2=Dense(units=256,activation='relu')
model.add(layer2)
layer3=Dense(units=128,activation='relu')
model.add(layer3)
output_layer=Dense(units=10,activation='softmax')
model.add(output_layer)

In [15]:
model.compile(optimizer='adagrad',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
model.fit(X_train,Y_train,epochs=100,batch_size=64,validation_data=(X_test,Y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 16s 313us/step - loss: 1.9914 - acc: 0.2541 - val_loss: 1.9072 - val_acc: 0.2959
Epoch 2/100
50000/50000 [==============================] - 14s 287us/step - loss: 1.8603 - acc: 0.3184 - val_loss: 1.8214 - val_acc: 0.3313
Epoch 3/100
50000/50000 [==============================] - 12s 237us/step - loss: 1.7985 - acc: 0.3447 - val_loss: 1.8059 - val_acc: 0.3453
Epoch 4/100
50000/50000 [==============================] - 12s 237us/step - loss: 1.7550 - acc: 0.3633 - val_loss: 1.7387 - val_acc: 0.3671
Epoch 5/100
50000/50000 [==============================] - 13s 263us/step - loss: 1.7198 - acc: 0.3755 - val_loss: 1.7085 - val_acc: 0.3797
Epoch 6/100
50000/50000 [==============================] - 14s 284us/step - loss: 1.6923 - acc: 0.3847 - val_loss: 1.6805 - val_acc: 0.3889
Epoch 7/100
50000/50000 [==============================] - 14s 283us/step - loss: 1.6694 - acc: 0.3933 - val_l

50000/50000 [==============================] - 18s 364us/step - loss: 1.3872 - acc: 0.5051 - val_loss: 1.4353 - val_acc: 0.4926
Epoch 59/100
50000/50000 [==============================] - 18s 365us/step - loss: 1.3849 - acc: 0.5060 - val_loss: 1.4480 - val_acc: 0.4938
Epoch 60/100
50000/50000 [==============================] - 18s 361us/step - loss: 1.3831 - acc: 0.5072 - val_loss: 1.4415 - val_acc: 0.4930
Epoch 61/100
50000/50000 [==============================] - 18s 362us/step - loss: 1.3810 - acc: 0.5065 - val_loss: 1.4290 - val_acc: 0.4992
Epoch 62/100
50000/50000 [==============================] - 18s 363us/step - loss: 1.3779 - acc: 0.5077 - val_loss: 1.4364 - val_acc: 0.4919
Epoch 63/100
50000/50000 [==============================] - 18s 362us/step - loss: 1.3763 - acc: 0.5098 - val_loss: 1.4273 - val_acc: 0.4979
Epoch 64/100
50000/50000 [==============================] - 18s 364us/step - loss: 1.3739 - acc: 0.5094 - val_loss: 1.4293 - val_acc: 0.4936
Epoch 65/100
50000/50000 [

In [21]:
print('training_score',model.evaluate(X_train,Y_train))
print('testing_score',model.evaluate(X_test,Y_test))

50000/50000 [==============================] - 4s 75us/step
training_score [1.3114090949630737, 0.5322]
10000/10000 [==============================] - 1s 75us/step
testing_score [1.4072744245529174, 0.5046]


In [22]:
Y_pred=model.predict(X_test)

In [23]:
Y_pred[0]

array([0.04701526, 0.08531091, 0.08515809, 0.38170993, 0.03613622,
       0.18310688, 0.09303533, 0.00331867, 0.0783288 , 0.00687986],
      dtype=float32)

In [24]:
Y_test[0]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])